In [1]:
import kaldiio
import numpy as np

def read_scp(scp_file):
    features = []
    keys = []
    with kaldiio.ReadHelper(f'scp:{scp_file}') as reader:
        for key, numpy_array in reader:
            features.append(numpy_array)
            keys.append(key)
            print(len(key))
    return np.array(features), keys

In [12]:
import pandas as pd

def read_labels(csv_file):
    df = pd.read_csv(csv_file)
    return df['Name'].values, df['Gender'].values

In [13]:
def match_features_labels(features, keys, label_keys, labels):
    key_to_index = {key: index for index, key in enumerate(keys)}
    matched_features = []
    matched_labels = []
    for key, label in zip(label_keys, labels):
        if key in key_to_index:
            matched_features.append(features[key_to_index[key]])
            matched_labels.append(label)
    return np.array(matched_features), np.array(matched_labels)


In [14]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from tqdm import tqdm

def train_and_test_svm(train_features, train_labels, test_features, test_labels):
    clf = svm.SVC()
    clf.fit(train_features, train_labels)
    
    # Use tqdm to show progress during prediction
    y_pred = []
    for i in tqdm(range(len(test_features)), desc="Predicting"):
        y_pred.append(clf.predict([test_features[i]])[0])
    
    accuracy = accuracy_score(test_labels, y_pred)
    return accuracy


: 

In [15]:
def main(train_scp, train_csv, test_scp, test_csv):
    train_features, train_keys = read_scp(train_scp)
    train_label_keys, train_labels = read_labels(train_csv)
    test_features, test_keys = read_scp(test_scp)
    test_label_keys, test_labels = read_labels(test_csv)

    train_features, train_labels = match_features_labels(train_features, train_keys, train_label_keys, train_labels)
    test_features, test_labels = match_features_labels(test_features, test_keys, test_label_keys, test_labels)

    accuracy = train_and_test_svm(train_features, train_labels, test_features, test_labels)
    print(f'Accuracy: {accuracy}')

if __name__ == '__main__':
    train_scp = '../Files/layer_0_features_train.scp'
    train_csv = '../Files/y_train.csv'
    test_scp = '../Files/layer_0_features_pfstar.scp'
    test_csv = '../Files/y_test.csv'
    main(train_scp, train_csv, test_scp, test_csv)


16
16
15
18
18
18
23
23
16
16
18
18
18
23
23
16
16
16
15
18
18
18
23
23
16
16
16
15
18
18
18
23
23
16
16
18
18
23
23
16
16
16
15
18
18
18
23
23
16
16
18
18
18
23
23
16
16
18
18
18
23
23
16
16
16
15
18
18
23
23
16
16
16
16
18
23
16
18
22
18
23
19
23
17
16
19
18
23
17
16
15
19
18
18
24
24
16
16
18
18
18
23
23
16
16
15
18
23
23
16
16
15
18
18
18
23
23
16
16
16
15
18
18
18
23
23
16
16
18
23
23
16
16
19
18
18
24
24
16
16
18
18
18
23
23
16
16
18
18
23
23
16
16
16
15
18
18
18
23
23
16
17
15
18
18
18
24
24
17
16
18
18
18
23
17
16
18
23
23
16
16
16
15
18
18
18
23
23
16
16
18
18
18
23
23
16
16
15
18
18
23
23
16
17
15
18
18
18
24
24
16
18
18
23
23
16
18
18
23
23
16
16
16
18
18
18
23
23
16
16
16
15
18
18
23
23
16
17
18
18
24
24
16
16
18
18
18
23
23
16
16
15
18
18
18
23
23
16
16
15
18
18
23
23
16
16
19
18
18
23
17
17
16
19
18
24
24
16
18
18
18
23
17
17
18
18
18
23
17
16
18
18
18
23
23
16
18
18
18
23
17
17
16
15
18
18
23
17
17
16
18
23
17
17
16
18
18
18
23
17
17
16
18
18
18
23
17
17
16
16
15
18
18
2

In [1]:
import kaldiio
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def read_scp(scp_file):
    features = []
    keys = []
    with kaldiio.ReadHelper(f'scp:{scp_file}') as reader:
        for key, numpy_array in reader:
            features.append(numpy_array.flatten())  # Flatten to 1D array
            keys.append(key)
    return np.array(features), keys

def read_labels(csv_file):
    df = pd.read_csv(csv_file)
    return df['Name'].values, df['Gender_bin'].values

def match_features_labels(features, keys, label_keys, labels):
    key_to_index = {key: index for index, key in enumerate(keys)}
    matched_features = []
    matched_labels = []
    for key, label in zip(label_keys, labels):
        if key in key_to_index:
            matched_features.append(features[key_to_index[key]])
            matched_labels.append(label)
    return np.array(matched_features), np.array(matched_labels)

def train_svm(train_features, train_labels):
    clf = svm.SVC()
    clf.fit(train_features, train_labels)
    return clf

def main(train_scp, train_csv, test_scp, test_csv):
    # Read features and labels
    train_features, train_keys = read_scp(train_scp)
    train_label_keys, train_labels = read_labels(train_csv)
    
    test_features, test_keys = read_scp(test_scp)
    test_label_keys, test_labels = read_labels(test_csv)

    # Match features and labels
    train_features, train_labels = match_features_labels(train_features, train_keys, train_label_keys, train_labels)
    test_features, test_labels = match_features_labels(test_features, test_keys, test_label_keys, test_labels)

    # Split train data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

    # Train SVM classifier
    clf = train_svm(X_train, y_train)

    # Predict on validation set
    val_predictions = clf.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_predictions)
    print(f'Validation Accuracy: {val_accuracy}')

    # Predict on test set
    test_predictions = clf.predict(test_features)
    test_accuracy = accuracy_score(test_labels, test_predictions)
    print(f'Test Accuracy: {test_accuracy}')

if __name__ == '__main__':
    train_scp = '../Files/layer_0_features_train.scp'
    train_csv = '../Files/y_train.csv'
    test_scp = '../Files/layer_0_features_pfstar.scp'
    test_csv = '../Files/y_test.csv'
    main(train_scp, train_csv, test_scp, test_csv)


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (856,) + inhomogeneous part.